# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [2]:
import pandas as pd

In [3]:
tic_tac_df = pd.read_csv('./tic-tac-toe.csv')

In [4]:
tic_tac_df.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [23]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np

In [30]:
encoder =  LabelEncoder()
encoded = tic_tac_df.apply(encoder.fit_transform)
ohe_df = pd.DataFrame(encoded)

ohe_df

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,2,2,2,2,1,1,2,1,1,1
1,2,2,2,2,1,1,1,2,1,1
2,2,2,2,2,1,1,1,1,2,1
3,2,2,2,2,1,1,1,0,0,1
4,2,2,2,2,1,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...
953,1,2,2,2,1,1,1,2,2,0
954,1,2,1,2,2,1,2,1,2,0
955,1,2,1,2,1,2,2,1,2,0
956,1,2,1,1,2,2,2,1,2,0


In [33]:
input_df = ohe_df[['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR']]
output_df = ohe_df['class']

output_df

0      1
1      1
2      1
3      1
4      1
      ..
953    0
954    0
955    0
956    0
957    0
Name: class, Length: 958, dtype: int64

In [34]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
final_input = sc.fit_transform(input_df)
final_input

array([[ 1.00322257,  1.08495342,  1.00322257, ...,  1.00322257,
        -0.16731812, -0.28682739],
       [ 1.00322257,  1.08495342,  1.00322257, ..., -0.28682739,
         1.08495342, -0.28682739],
       [ 1.00322257,  1.08495342,  1.00322257, ..., -0.28682739,
        -0.16731812,  1.00322257],
       ...,
       [-0.28682739,  1.08495342, -0.28682739, ...,  1.00322257,
        -0.16731812,  1.00322257],
       [-0.28682739,  1.08495342, -0.28682739, ...,  1.00322257,
        -0.16731812,  1.00322257],
       [-0.28682739, -0.16731812,  1.00322257, ..., -0.28682739,
         1.08495342,  1.00322257]])

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [35]:
from sklearn.model_selection import train_test_split


final_input_train, final_input_test, output_df_train, output_df_test = train_test_split(final_input, output_df)
display(final_input_train.shape)
display(output_df_train.shape)

(718, 9)

(718,)

In [21]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [54]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation="relu", name="layer1"))
model.add(tf.keras.layers.Dense(128, activation="relu", name="layer2"))
model.add(tf.keras.layers.Dense(60, activation="relu", name="layer3"))
model.add(tf.keras.layers.Dense(30, activation="relu", name="layer4"))
model.add(tf.keras.layers.Dense(15, activation="relu", name="layer5"))
model.add(tf.keras.layers.Dense(2, activation=tf.nn.softmax))


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(final_input_train, output_df_train, epochs=8)

Epoch 1/8
23/23 [==============================] - 0s 5ms/step - loss: 0.6333 - accuracy: 0.6490
Epoch 2/8
23/23 [==============================] - 0s 6ms/step - loss: 0.5724 - accuracy: 0.7075
Epoch 3/8
23/23 [==============================] - 0s 6ms/step - loss: 0.5069 - accuracy: 0.7479
Epoch 4/8
23/23 [==============================] - 0s 8ms/step - loss: 0.4014 - accuracy: 0.8552
Epoch 5/8
23/23 [==============================] - 0s 6ms/step - loss: 0.3097 - accuracy: 0.8760
Epoch 6/8
23/23 [==============================] - 0s 6ms/step - loss: 0.2343 - accuracy: 0.9123
Epoch 7/8
23/23 [==============================] - 0s 7ms/step - loss: 0.1746 - accuracy: 0.9387
Epoch 8/8
23/23 [==============================] - 0s 7ms/step - loss: 0.1203 - accuracy: 0.9610


In [55]:
val_loss, val_acc = model.evaluate(final_input_test, output_df_test)
print(val_loss)
print(val_acc)

8/8 [==============================] - 0s 9ms/step - loss: 0.3060 - accuracy: 0.8833
0.30601808428764343
0.8833333253860474


In [56]:
model.save('tic-tac-toe.model')

INFO:tensorflow:Assets written to: tic-tac-toe.model/assets


## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [57]:
new_model = tf.keras.models.load_model('tic-tac-toe.model')

predictions = new_model.predict(final_input_train)

In [58]:
output_df_test.reset_index(inplace=True, drop=True)

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [63]:
new_model_2 = tf.keras.models.load_model('tic-tac-toe.model')
optimize = tf.keras.optimizers.Adam(learning_rate=0.0015)
new_model_2.compile(optimizer=optimize,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

new_model.fit(final_input_train, output_df_train, epochs=5)

Epoch 1/5
23/23 [==============================] - 0s 6ms/step - loss: 0.0933 - accuracy: 0.9708
Epoch 2/5
23/23 [==============================] - 0s 6ms/step - loss: 0.0588 - accuracy: 0.9875
Epoch 3/5
23/23 [==============================] - 0s 5ms/step - loss: 0.0298 - accuracy: 0.9958
Epoch 4/5
23/23 [==============================] - 0s 5ms/step - loss: 0.0239 - accuracy: 0.9972
Epoch 5/5
23/23 [==============================] - 0s 4ms/step - loss: 0.0080 - accuracy: 1.0000


In [64]:
val_loss, val_acc = new_model_2.evaluate(final_input_test, output_df_test)
print(val_loss)
print(val_acc)

8/8 [==============================] - 0s 6ms/step - loss: 0.3060 - accuracy: 0.8833
0.30601808428764343
0.8833333253860474


**Which approach(es) did you find helpful to improve your model performance?**

Adding more layers did not get better results, but changing parameters did not